In [1]:
import semiolog as slg

semiotic = slg.Cenematic("en_bnc_test")

# semiotic.corpus.build(
#     save = True,
#     )

# semiotic.vocab.build(
#     save = True,
#     parallel = True,
#     )

# semiotic.paradigmatic.build(load_tokenized=True, save_tokenized=True, n_sents=10000)

SLG: Checking config correctness...
SLG: Config correct!


100%|██████████| 3/3 [00:00<00:00, 467.80it/s]

SLG Warning: models/en_bnc_test/vocabulary/merges.txt does not exist.
Vocabulary will not be loaded from file.



In [2]:
semiotic.vocab.build(
    save = True,
    parallel = True,
    )

SLG: Computing in parallel. CPU count: 4
SLG: Starting parallel jobs.
SLG: Normalize and jobs data...


  0%|          | 0/67796 [00:00<?, ?ex/s]


  0%|          | 246/67796 [00:00<00:28, 2411.25ex/s]





  1%|          | 488/67796 [00:00<00:33, 2023.50ex/s]


  1%|▏         | 885/67796 [00:00<00:39, 1696.71ex/s]


  2%|▏         | 1096/67796 [00:00<00:36, 1824.06ex/s]


  2%|▏         | 1345/67796 [00:00<00:32, 2027.99ex/s]


  2%|▏         | 1553/67796 [00:00<00:35, 1867.40ex/s]


  3%|▎         | 1745/67796 [00:00<00:38, 1713.75ex/s]


  3%|▎         | 1983/67796 [00:01<00:34, 1889.39ex/s]

  3%|▎         | 2178/67796 [00:01<00:35, 1858.14ex/s]


  3%|▎         | 2368/67796 [00:01<00:37, 1734.89ex/s]


  4%|▍         | 2569/67796 [00:01<00:36, 1808.92ex/s]

  4%|▍         | 2754/67796 [00:01<00:41, 1554.93ex/s]


  4%|▍         | 2970/67796 [00:01<00:37, 1706.85ex/s]


  5%|▍         | 3156/67796 [00:01<00:38, 1668.01ex/s]


  5%|▍         | 3329/67796 [00:01<00:42, 1527.94ex/s]


  5%|▌         | 3524/67796 [00:02<00:39, 1636.00ex/s]


  5%|▌         | 3694/67796 [00:02<00:39, 1621

... computed in 44.29777812957764 secs.

SLG: Build alphabet...
... computed in 0.0005888938903808594 secs.

SLG: Alphabet Size: 122
SLG: Special Tokens Size: 5
SLG: Terms to compute: 1873

SLG: Enter loop


  0%|          | 0/1873 [00:00<?, ?it/s]

SLG: Compute freq...
... computed in 4.539822101593018 secs.

SLG: Vocabulary built

SLG: Vocabulary saved to models/en_bnc_test/vocabulary

SLG: Syntagmatic and Paradigmatic updated with the new vocabulary



In [3]:
from random import randint
n = randint(0,semiotic.corpus.train.num_rows)
# n = 51900
n= 264637
sent = semiotic.corpus.train[n]["text"]+"]["

sent_seq = semiotic(sent)
print(n)
sent_seq.tree.plot()

264637


interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function semiolog.syntagmatic.tree.Tree.plot.<locals>.inter()>

In [8]:
sent_seq.parad_chain.df()

,an_0,umberof_1,americ_2,an_3,compan_4,ies_5,in_6,addition_7,to_8,those_9,...,s_18,imilar_19,approach_20,es_21,ho_22,er_23,ret_24,al_25,198_26,6_27
0,fore,tofthe,ived,ived,ok,ever,vot,fore,str,fore,...,esp,ok,ever,fore,ever,fore,icular,ises,fore,ived
1,s,remain,ace,ok,vot,across,atleast,ately,ived,contr,...,ived,descri,fore,ived,ived,great,upto,198,atleast,icular
2,ever,ever,ises,erson,ef,ok,med,idnt,ace,ever,...,ig,atleast,upto,icular,ok,aver,emp,apro,vot,ok
3,str,child,ever,century,aul,play,fore,red,th,erson,...,erson,ived,irm,upto,ises,vot,aver,s,ever,6
4,ived,hop,fore,ld,upto,k,ok,aand,although,atleast,...,med,ever,canbe,aver,thatwas,thed,ived,ewould,suc,two
5,ok,ived,ister,fore,fore,een,str,esp,fore,child,...,ook,str,atleast,sw,upto,str,fore,erson,ived,emp
6,icular,dat,str,ů,aver,hop,irm,erc,yeah,ants,...,atleast,s,approach,ok,aver,front,atleast,contr,ok,fore
7,aver,withthe,ingand,ual,does,emp,ever,ived,irm,vot,...,il,swith,jects,str,irm,thats,iffer,care,ait,dat
8,forc,party,ict,list,list,s,s,irm,himself,om,...,univers,ery,por,op,plac,erson,ok,thats,again,west
9,erson,thepr,pound,fre,care,there,forc,ok,ery,ects,...,ify,row,ef,sas,bank,ived,report,ever,op,ewould
